<a href="https://colab.research.google.com/github/rosonacharya/ChatGpt-Sentiment-Analysis/blob/main/Notebook/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Go to project folder
import os
os.chdir('/content/drive/MyDrive/ChatGpt-Sentiment-Analysis/')

Mounted at /content/drive


# **Import Libraries**

In [5]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **Loading the dataframe**

In [6]:
#reading the dataframe
df = pd.read_csv("Data/chatgpt.csv")
print(df.head())

   Unnamed: 0                                             tweets   labels
0           0  ChatGPT: Optimizing Language Models for Dialog...  neutral
1           1  Try talking with ChatGPT, our new AI system wh...     good
2           2  ChatGPT: Optimizing Language Models for Dialog...  neutral
3           3  THRILLED to share that ChatGPT, our new model ...     good
4           4  As of 2 minutes ago, @OpenAI released their ne...      bad


In [7]:
#droping the unnamed columns
df.drop(columns=['Unnamed: 0'],inplace=True)

In [8]:
#Checking if column is drop or not
print(df)

                                                   tweets   labels
0       ChatGPT: Optimizing Language Models for Dialog...  neutral
1       Try talking with ChatGPT, our new AI system wh...     good
2       ChatGPT: Optimizing Language Models for Dialog...  neutral
3       THRILLED to share that ChatGPT, our new model ...     good
4       As of 2 minutes ago, @OpenAI released their ne...      bad
...                                                   ...      ...
219289  Other Software Projects Are Now Trying to Repl...      bad
219290  I asked #ChatGPT to write a #NYE Joke for SEOs...     good
219291  chatgpt is being disassembled until it can onl...      bad
219292  2023 predictions by #chatGPT. Nothing really s...      bad
219293   From ChatGPT, neat stuff https://t.co/qjjUF2Z2m0  neutral

[219294 rows x 2 columns]


In [9]:
#Viewing the first 5  rows in tweets column
def preview_tweets(df, n=5, column='tweets'):
    for tweet in df[column].head(n):
        print(tweet)

In [10]:
preview_tweets(df)

ChatGPT: Optimizing Language Models for Dialogue https://t.co/K9rKRygYyn @OpenAI
Try talking with ChatGPT, our new AI system which is optimized for dialogue. Your feedback will help us improve it. https://t.co/sHDm57g3Kr
ChatGPT: Optimizing Language Models for Dialogue https://t.co/GLEbMoKN6w #AI #MachineLearning #DataScience #ArtificialIntelligence\n\nTrending AI/ML Article Identified &amp; Digested via Granola; a Machine-Driven RSS Bot by Ramsey Elbasheer https://t.co/RprmAXUp34
THRILLED to share that ChatGPT, our new model optimized for dialog, is now public, free, and accessible to everyone. https://t.co/dyvtHecYbd https://t.co/DdhzhqhCBX https://t.co/l8qTLure71
As of 2 minutes ago, @OpenAI released their new ChatGPT. \n\nAnd you can use it right now 👇 https://t.co/VyPGPNw988 https://t.co/cSn5h6h1M1


 # **Removing HTML Tags**

In [11]:
import re

In [12]:
def clear_social_text(text: str) -> str:
    text = str(text).lower()

    text = re.sub(r'http\S+|www\S+', '', text)   # Remove URLs (http, https, www)


    text = re.sub(r'@\w+', '', text)  # Remove mentions (@username)


    text = re.sub(r'#\w+', '', text) # Remove hashtags (#hashtag)


    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespaces

    return text



In [13]:
df['tweets'] = df['tweets'].apply(clear_social_text)

In [14]:
preview_tweets(df)

chatgpt: optimizing language models for dialogue
try talking with chatgpt, our new ai system which is optimized for dialogue. your feedback will help us improve it.
chatgpt: optimizing language models for dialogue \n\ntrending ai/ml article identified &amp; digested via granola; a machine-driven rss bot by ramsey elbasheer
thrilled to share that chatgpt, our new model optimized for dialog, is now public, free, and accessible to everyone.
as of 2 minutes ago, released their new chatgpt. \n\nand you can use it right now 👇


# **Remove the non-printing characters from text**

In [15]:
import string
import re
from collections import Counter

Checking if there is non printing characters or not

In [16]:
#Checking if there is non printing characters or not
def has_non_printing(text: str) -> bool:
    return any(ch not in string.printable for ch in text)

In [17]:
df['has_non_printing'] = df['tweets'].apply(has_non_printing)

df[df['has_non_printing']].head()

,tweets,labels,has_non_printing
4,"as of 2 minutes ago, released their new chatgp...",bad,True
17,models are set to become the search engines of...,bad,True
28,chatgpt: optimizing language models for dialog...,neutral,True
34,openai's new chatgpt is very honest 😀,good,True
36,ok so 's new can basically just generate promp...,neutral,True


Listing  non-printing characters found in text

In [18]:
#listing  non-printing characters found in text
def extract_non_printing(text: str):
    return [ch for ch in str(text) if ch not in string.printable]

In [19]:
non_printing_chars = Counter()
for text in df['tweets']:
    non_printing_chars.update(extract_non_printing(text))

In [20]:
#Viewing the top common non printing chars
non_printing_chars.most_common(10)

[('’', 30896),
 ('…', 7643),
 ('“', 6406),
 ('”', 6178),
 ('😂', 3590),
 ('️', 3497),
 ('🤯', 3494),
 ('—', 3087),
 ('‘', 2307),
 ('👇', 2289)]

In [21]:
for ch, count in non_printing_chars.most_common(10):
    print(f"U+{ord(ch):04X} | {repr(ch)} | count={count}")

U+2019 | '’' | count=30896
U+2026 | '…' | count=7643
U+201C | '“' | count=6406
U+201D | '”' | count=6178
U+1F602 | '😂' | count=3590
U+FE0F | '️' | count=3497
U+1F92F | '🤯' | count=3494
U+2014 | '—' | count=3087
U+2018 | '‘' | count=2307
U+1F447 | '👇' | count=2289


Targeted cleaning (only remove what exists)

In [22]:
def remove_known_non_printing(text: str):
    for ch in non_printing_chars:
        text = text.replace(ch, '')
    return text

In [23]:
df['tweets'] = df['tweets'].astype(str).apply(remove_known_non_printing)

In [24]:
df['tweets'].apply(has_non_printing).value_counts()

,count
tweets,
False,219294


# **Removing punctuation,stopwords, whitespace and numbers**

Importing Libraries

In [25]:
import nltk
from nltk.corpus import stopwords
import re

In [26]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
# Set of English stopwords
STOP_WORDS = set(stopwords.words('english'))
PUNCT_REGEX = re.compile(f"[{re.escape(string.punctuation)}]") #remove punctuation
WHITESPACE_REGEX = re.compile(r'\s+') #remove whitespace
NUMBER_REGEX = re.compile(r'\d+') #remove number

In [28]:
def clean_tweet(text: str) -> str:
    text = str(text).lower()
    text = PUNCT_REGEX.sub("", text)
    text = NUMBER_REGEX.sub("", text)
    text = " ".join(word for word in text.split() if word not in STOP_WORDS)
    text = WHITESPACE_REGEX.sub(" ", text).strip()
    return text

In [29]:
df['tweets'] = df['tweets'].apply(clean_tweet)

In [30]:
preview_tweets(df)

chatgpt optimizing language models dialogue
try talking chatgpt new ai system optimized dialogue feedback help us improve
chatgpt optimizing language models dialogue nntrending aiml article identified amp digested via granola machinedriven rss bot ramsey elbasheer
thrilled share chatgpt new model optimized dialog public free accessible everyone
minutes ago released new chatgpt nnand use right


# **Emoji handling**

In [31]:
import re

In [33]:
# Emoji regex pattern
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           "]+", flags=re.UNICODE)

# Detect which tweets contain emojis
df['has_emoji'] = df['tweets'].apply(lambda x: bool(emoji_pattern.search(str(x))))

# Count how many tweets have emojis
print("Tweets containing emojis:", df['has_emoji'].sum())


Tweets containing emojis: 0


# **Handling repeated characters**

In [37]:
def detect_and_normalize_repeats(text, max_repeat=2):

    pattern = re.compile(r'(.)\1{2,}')
    if pattern.search(str(text)):  # detect first
        text = pattern.sub(r'\1'*max_repeat, str(text))  # normalize
    return text



In [39]:
df['tweets'] = df['tweets'].apply(detect_and_normalize_repeats)

In [40]:
repeat_pattern = re.compile(r'(.)\1{2,}')

# Detect any remaining repeated characters
df['has_repeats'] = df['tweets'].apply(lambda x: bool(repeat_pattern.search(str(x))))

# Count how many tweets still have repeated characters
print("Tweets still containing repeated characters:", df['has_repeats'].sum())

Tweets still containing repeated characters: 0


# **Word Lemmatizer**

In [41]:
import nltk
from nltk.stem import WordNetLemmatizer

In [42]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [43]:
lemmatizer = WordNetLemmatizer()

In [44]:
def lemmatize_tweet(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Split text into words, lemmatize each, join back into a string
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [45]:
df['tweets'] = df['tweets'].apply(lemmatize_tweet)

In [46]:
preview_tweets(df)

chatgpt optimizing language model dialogue
try talking chatgpt new ai system optimized dialogue feedback help u improve
chatgpt optimizing language model dialogue nntrending aiml article identified amp digested via granola machinedriven rss bot ramsey elbasheer
thrilled share chatgpt new model optimized dialog public free accessible everyone
minute ago released new chatgpt nnand use right


# **POS Aware Lemmatization**

In [47]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

In [52]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [53]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

In [54]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(text: str) -> str:
    if not isinstance(text, str):
        return ""

    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    lemmas = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in pos_tags
    ]

    return " ".join(lemmas)


In [55]:
df["tweets"] = df["tweets"].apply(lemmatize_tweet)

In [56]:
preview_tweets(df)

chatgpt optimize language model dialogue
try talk chatgpt new ai system optimize dialogue feedback help u improve
chatgpt optimize language model dialogue nntrending aiml article identify amp digested via granola machinedriven rss bot ramsey elbasheer
thrill share chatgpt new model optimize dialog public free accessible everyone
minute ago release new chatgpt nnand use right


# **Tokenization**

In [57]:
import nltk
from nltk.tokenize import word_tokenize

In [58]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [59]:
def tokenize_tweet(text: str) -> list:
    if not isinstance(text, str):
        return []

    return word_tokenize(text)

In [60]:
df['tweet_tokens'] = df['tweets'].apply(tokenize_tweet)

# **Saving to new csv**

In [61]:
print(df.head())

                                              tweets   labels  \
0           chatgpt optimize language model dialogue  neutral   
1  try talk chatgpt new ai system optimize dialog...     good   
2  chatgpt optimize language model dialogue nntre...  neutral   
3  thrill share chatgpt new model optimize dialog...     good   
4     minute ago release new chatgpt nnand use right      bad   

   has_non_printing  has_emoji  has_repeats  \
0             False      False        False   
1             False      False        False   
2             False      False        False   
3             False      False        False   
4              True      False        False   

                                        tweet_tokens  
0     [chatgpt, optimize, language, model, dialogue]  
1  [try, talk, chatgpt, new, ai, system, optimize...  
2  [chatgpt, optimize, language, model, dialogue,...  
3  [thrill, share, chatgpt, new, model, optimize,...  
4  [minute, ago, release, new, chatgpt, nnand, us..

In [62]:
print(df.columns.to_list())

['tweets', 'labels', 'has_non_printing', 'has_emoji', 'has_repeats', 'tweet_tokens']


In [63]:
df_to_save = df.drop(columns=['has_non_printing','has_non_printing', 'has_emoji', 'has_repeats'])

In [65]:
print(df_to_save.columns.to_list())

['tweets', 'labels', 'tweet_tokens']


In [66]:
df_to_save.to_csv('Data/chatgpt_clean.csv', index=False)